# Car Accidents Severity Prediction using Machine Learning




#### By: Ahmad Khairi Bin Ahmad Khir

### Business Understanding

Multiple of factors and conditions can influence the severity of a car accident. With the help of an algorithm, the prediction of severity of an accident can be done with the help of variable such as weather condition, road condition, light condition and so on. Using this data, a model can be construct to predict the severity of an accident bound to happen based on the past historical dataset that contains the attribute that factors to an accident. Thus, in order to reduce the frequency of accidents in a certain area, machine learning can be apply to these problem which will helps the local government or GPS service provider to determine which sort of conditions and factors may contributed to a car accidents and exercise necessary action to minimize the probability of a road user from being involved in a car accidents.

### Data Understanding

In this project, we used the datasets from the Seattle SDOT Traffic Management Division which titled Collisions – All years. We cleaned the datasets by using the ‘SEVERITYCODE’ column as our target variable with degree of 0 to 5 in terms of severity. The predictor variables or features used in this project are the ‘JUNCTIONTYPE’, ‘WEATHER’, ‘ROADCOND’, ‘LIGHTCOND’ and ‘SPEEDING’. The features which of originally datatype object were converted into datatype numerical for analysis and also the dataset were balanced which the disparity between the severity categories may resulted in inaccurate prediction by the model.